#Примитивы синхронизации потоков

Одной из особенностей при написании многопоточных программ является состояние гонки. Это состояние возникает, когда два или более потоков обращаются к общим ресурсам. 

Например, мы имеем базу данных, в одной из ячеек которой лежит число, допустим, 0. Запуская два потока одновременно через ThreadPoolExecutor, мы запускаем функцию, которая считывает число, увеличивает его на 1 и записывает в базу данных.

In [2]:
class DataBase:
  def __init__(self):
    self.value = 0

  def update(self, name):
    print(name, " — start thread")
    local_copy = self.value
    local_copy += 1
    time.sleep(0.1)
    self.value = local_copy
    print(name, " — finish thread")

Мы ожидаем, что первый поток запишет 1, а второй впоследствии увеличит 1 до 2. Но т.к. обращение происходит практически одновременно (вспомним GIL), в результате так и останется 1. Так как оба потока считают 0 в качестве исходных данных. 

Чтобы успешно разрешить состояние гонки, в Python добавлены примитивы синхронизации, которые многим могут быть знакомы из других языков программирования.

__Lock — это блокировка, которая может одновременно удерживаться только одним потоком.__

Если другой поток хочет получить доступ к lock, который занял другой поток, то он будет вынужден ждать, пока занявший поток не отпустит блокировку.

In [5]:
import threading

lock = threading.Lock()
lock.acquire()  # Выполнить блокировку данного участка кода
#Some coding#
lock.release()

Что удобно, можно использовать контекстный менеджер и не беспокоиться о необходимости выполнять явную блокировку / разблокировку.

In [6]:
with self._lock:
  locl_copy = self.value
  local_copy += 1
  time.sleep(0.1)
  self.value = local_copy

Семафор чем-то похож на Lock с той лишь разницей, что в него встроен счетчик, который блокирует доступ в случае превышения числа потоков, которые удерживают семафор. 

In [ ]:
import threading

max_connections = 10
semaphore = threading.BoundedSemaphore(max_connections)
semaphore.acquire() #уменьшает счетчик на 1
"""Доступ к общим ресурсам"""
semaphore.release()

С каждым acquire() счетчик уменьшается, с release() — увеличивается, но когда счетчик равен 0, новый поток будет вынужден ждать, пока не освободится место для него.

Так же во время разработки программы с многопоточностью удобно использовать модуль Queue, который реализует механизм очередей с поддержкой threadsafe. Это означает, что, используя очередь, можно безопасно обмениваться информацией между потоками.

In [7]:
import threading
import time
from queue import Queue
from threading import Thread

num_threads = 2
def do_work(item):
  time.sleep(1)
  print("My task is ", item, " I am ", threading.current_thread())

def worker():
  while True:
    item = q.get()
    print("got task — ", item)
    do_work(item)
    q.task_done()

q = Queue()
for i in range(num_threads):
  t = Thread(target=worker)
  t.setDaemon(True)
  t.start()

for i in range(0, 5):
  q.put(i)

q.join()

got task —  got task —  1
0
My task is My task is  0  I am  <Thread(Thread-11, started daemon 139946279646976)>
got task —  2
 1  I am  <Thread(Thread-12, started daemon 139946226910976)>
got task —  3
My task is  2  I am  <Thread(Thread-11, started daemon 139946279646976)>
got task —  4
My task is  3  I am  <Thread(Thread-12, started daemon 139946226910976)>
My task is  4  I am  <Thread(Thread-11, started daemon 139946279646976)>


In [ ]:
get task - 0
get task - 1
my task is 0 i am <Thread(Thread-1, started daemon 123145343508480)>
get task - 2
my task is 1 i am <Thread(Thread-2, started daemon 123145348763648)>
get task - 3
my task is 2 i am <Thread(Thread-1, started daemon 123145343508480)>
get task - 4
my task is 3 i am <Thread(Thread-2, started daemon 123145348763648)>
my task is 4 i am <Thread(Thread-1, started daemon 123145343508480)>

В данном примере мы запускаем несколько потоков, создаем очередь и помещаем в нее задания. Потоки, используя безопасный метод q.get(), получают задания и выполняют их. 

На таком примере можно легко реализовать собственный скрипт для скачивания сайтов по доступным urls или приложение, которое после обсчета некоторых данных отдаст задачу на добавление результатов в базу данных.